In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/dataset

In [ ]:
!ls -al "/content/drive/MyDrive/Colab Notebooks/dataset/"

total 472385
-rw------- 1 root root 172964152 Dec 24 07:24 best_model.h5
-rw------- 1 root root  62685307 Dec 24 06:51 flower_prepared.zip
-rw------- 1 root root 124090252 Dec 29 07:49 galucoma.zip
-rw------- 1 root root 123981705 Dec 29 06:40 raw_galucoma.zip


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/dataset/galucoma.zip" -d /content/dataset

In [ ]:
# tf.keras.applications.inception_v3.preprocess_input

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Input

input_tensor = Input(shape=(224, 224, 3))
model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

96124928/96112376 [==============================] - 4s 0us/step


In [ ]:
from tensorflow.keras.applications import InceptionV3

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img , img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
%matplotlib inline

BATCH_SIZE = 64
FILE_DIR = "/content/dataset/processed_data/"


preprocessor = preprocess_input

train_data_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=preprocessor,
    #   validation_split= 0.2,
).flow_from_directory(
      FILE_DIR+"train",
      target_size=(240,240),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
)

valid_data_generator = ImageDataGenerator(
 preprocessing_function=preprocessor
).flow_from_directory(
      FILE_DIR+ "valid",
      target_size=(240,240),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
     
)

test_data_generator = ImageDataGenerator(
    preprocessing_function=preprocessor

).flow_from_directory(
      FILE_DIR+ "test",
      target_size=(240,240),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
      
)


Found 1230 images belonging to 3 classes.
Found 160 images belonging to 3 classes.
Found 154 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,BatchNormalization, Dropout,Conv2D, MaxPool2D, Activation, Input
from tensorflow import keras
from tensorflow.keras.initializers import GlorotNormal




initializer = GlorotNormal()
# from keras import backend as K

# 선행학습된 기준모델을 만듭니다
preprocessor = preprocess_input


conv_layers = InceptionV3(include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(240,240,3),
    pooling=None,
    classes=1000)
    #classifier_activation="softmax")

conv_layers.trainable = False

model = keras.Sequential()

# model.add(conv_layers)
# model.add(Input(input_shape = (240, 240, 3)))
model.add(Conv2D(16, (20, 20),activation='relu', kernel_initializer=initializer)) #,input_shape=(240, 240, 3)
model.add(Dropout(0.5))
model.add(MaxPool2D((2,2),strides=2))
model.add(Conv2D(32, (40, 40),activation='relu', kernel_initializer=initializer)) # ,input_shape=(110, 110, 16)
model.add(Dropout(0.5))
model.add(MaxPool2D((2,2),strides=2))
model.add(Flatten())
model.add(Dense(32,activation='relu',kernel_initializer=initializer))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu',kernel_initializer=initializer))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer="Adagrad", metrics=['acc'])


In [ ]:
history = model.fit(
      train_data_generator,
      validation_data=valid_data_generator,
      epochs=10
)

Epoch 1/10
 5/20 [======>.......................] - ETA: 17s - loss: 1.1671 - acc: 0.3556

KeyboardInterrupt: ignored

In [ ]:
model.summary()

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB2

preprocessor = efficientnet.preprocess_input

# conv_layers = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# conv_layers.trainable = False

model = keras.Sequential()

# model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
model.summary()




plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()


loss, acc = model.evaluate(test_data_generator)
print("loss=", loss)
print("acc=", acc)



batch_x, batch_y = test_data_generator.next()
y_ = model.predict(batch_x)
predicted = np.argmax(y_, axis=-1)

plt.plot(batch_y[:100], "o")
plt.plot(predicted[:100], '.')
plt.show()


custom_labels = list(test_data_generator.class_indices.keys())
print("label category index =", batch_y[0])
print("predicted category index =", predicted[0])
print("predicted category name =", custom_labels[predicted[0]])
